In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import shutil

In [10]:
source_directory = '../Data_Q2/train_data'
folders = ['Cloudy', 'Foggy', 'Rainy', 'Snowy', 'Sunny']

# 检查是否已存在这些子文件夹，如果不存在则在source_directory中创建它们
for folder_name in folders:
    folder_path = os.path.join(source_directory, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# 将数据移动到相应的子文件夹中
for filename in os.listdir(source_directory):
    if os.path.isfile(os.path.join(source_directory, filename)):  # 确保只移动文件而不是文件夹
        for folder_name in folders:
            if folder_name in filename:
                source_path = os.path.join(source_directory, filename)
                destination_path = os.path.join(source_directory, folder_name, filename)
                shutil.move(source_path, destination_path)

In [14]:
# Data directory
data_dir = '../Data_Q2/train_data'

# Image size and batch size settings
image_size = (224, 224)
batch_size = 32

# Initialize ImageDataGenerator with rescaling
datagen = ImageDataGenerator(rescale=1./255)

# Load images from directory
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')


Found 250 images belonging to 5 classes.


In [20]:
def build_vgg16(input_shape=(224, 224, 3), num_classes=5):
    # Define the input
    input_tensor = Input(shape=input_shape)

    # First block
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Second block
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Third block
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fourth block
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Fifth block
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    # Flatten and Dense layers
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=x)
    return model

In [21]:
# Build the model
model = build_vgg16()

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                             

In [22]:
# Train the model
history = model.fit(train_generator, epochs=50)

Epoch 1/50
8/8 [==============================] - 47s 6s/step - loss: 1.6115 - accuracy: 0.1760
Epoch 2/50
8/8 [==============================] - 46s 6s/step - loss: 1.6090 - accuracy: 0.2040
Epoch 3/50
8/8 [==============================] - 46s 6s/step - loss: 1.6026 - accuracy: 0.1960
Epoch 4/50
8/8 [==============================] - 47s 6s/step - loss: 1.5955 - accuracy: 0.2240
Epoch 5/50
8/8 [==============================] - 47s 6s/step - loss: 1.5414 - accuracy: 0.3280
Epoch 6/50
8/8 [==============================] - 47s 6s/step - loss: 1.4613 - accuracy: 0.4560
Epoch 7/50
8/8 [==============================] - 47s 6s/step - loss: 1.3375 - accuracy: 0.4840
Epoch 8/50
8/8 [==============================] - 47s 6s/step - loss: 1.1167 - accuracy: 0.5240
Epoch 9/50
8/8 [==============================] - 46s 6s/step - loss: 1.0209 - accuracy: 0.5520
Epoch 10/50
8/8 [==============================] - 44s 6s/step - loss: 1.1494 - accuracy: 0.5240
Epoch 11/50
8/8 [======================

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the training set
train_loss, train_accuracy = model.evaluate(train_generator)
print("Accuracy on training set:", train_accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Plot the training accuracy and loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.title('Training Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.title('Training Loss')
plt.legend()
plt.show()
